Hunter Bennett  
Christopher K Glass Laboratory  
Created 20180103 | Last updated 20180103
_____

Here we want to use a more complex differential calling software (DESeq2, edgeR) to analyze the differential peaks in our ATAC Seq Data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
import subprocess
import os
import sys
import glob
import re

# import user defined packages
sys.path.insert(0, '/home/h1bennet/scripts')
import hbUtils

# plot matplotlib plots in notebook
%matplotlib inline

In [2]:
# define path to data
datapath = {'kupffer':'/data/mm10/Kupffer/ATAC/'}

# define output directory
outdir = '/home/h1bennet/liverStrains/results/180103_ATAC/'

if not os.path.isdir(outdir):
    subprocess.call(['mkdir', outdir])

# define samples for both whole liver and kupffer
samples = {'kupffer':["aj_Kupffer_ATAC_AMLNDiet_30week_AJ3A_JSS_TDT_16_09_26",
"aj_Kupffer_ATAC_AMLNDiet_30week_AJ3B_JSS_TDT_16_09_26",
"aj_Kupffer_ATAC_ControlDiet_30week_AJ1B_JSS_TDT_16_09_26",
"aj_Kupffer_ATAC_ControlDiet_30week_AJ1C_JSS_TDT_16_09_28",
"balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3C_JSS_TDT_16_09_26",
"balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3D_JSS_TDT_16_09_28",
"balbc_Kupffer_ATAC_ControlDiet_30week_Balb1A_JSS_TDT_16_09_26",
"balbc_Kupffer_ATAC_ControlDiet_30week_Balb1B_JSS_TDT_16_09_26",
"NCoRWT_KupfferTotal_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_16_09_19",
"NCoRWT_KupfferTotal_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_16_09_21",
"NCoRWT_KupfferTotal_ATAC_ControlDiet_30week_LN134B_JSS_TDT_16_09_21",
"NCoRWT_KupfferTotal_ATAC_ControlDiet_30week_LN134D_JSS_TDT_16_09_21"]}

bad_samples = {'LN140D': 'cancer', 'LN136B': 'cancer', 'LN148B': 'cancer',
              'LN144A': 'cancer', 'LN144C': 'cancer', 'LN182B': 'cancer',
              'LN182B': 'cancer', 'LN203B': 'cancer', 'BALB3A': 'splenomegaly',
              'BALB4D': 'hyper-fibrosis', 'LN148B': 'cancer', 'LN166A': 'cancer'}

In [3]:
# user defined functions
def merge_peaks(peak1, peak2, outfile, dist='given', print_err=False):
    import subprocess
    import os
    # construct function call
    mP_call = ["mergePeaks"]
    mP_call.extend(['-d', dist])
    mP_call.extend([peak1,
                    peak2])
    # call merge peaks
    if not os.path.isfile(outfile):
        print('calling mergePeaks...')
        p = subprocess.Popen(mP_call,
                            stdout = subprocess.PIPE,
                            stderr = subprocess.PIPE)
        output, err = p.communicate() #in bytes
        if print_err: print(err.decode('utf-8'))
        f = open(outdir + '/' + name + '_merge.txt', 'w')
        f.write(output.decode("utf-8"))
        f.close()
    else: print('merge peaks file ' + outdir + '/optimal_peakfile_merge.txt already exists!')
    return outdir + '/' + name + '_merge.txt'

def anno_peaks(dir1, dir2, outdir, dist='given', noann=False, nogene=False, print_err=False):
    import subprocess
    import os
    # call merge peaks
    mPPath = merge_peaks(dir1, dir2, outdir, dist, print_err=print_err)
    # construct function call
    aPCallList = ['annotatePeaks.pl', mPPath, 'mm10']
    if noann: aPCallList.extend(['-noann'])
    if nogene: aPCallList.extend(['-nogene'])
    aPCallList.extend(['-d', dir1 + '/pooled_tag_dirs', dir2 + '/pooled_tag_dirs'])
    # call annotate peaks
    aPPath = mPPath.replace('.txt', '_anno.txt')
    if not os.path.isfile(aPPath):
        print('calling annotatePeaks...')
        p = subprocess.Popen(aPCallList,
                         stdout=subprocess.PIPE,
                         stderr=subprocess.PIPE)
        output, err = p.communicate()
        if print_err: print(err.decode('utf-8'))
        #write output
        f = open(aPPath, 'w')
        f.write(output.decode("utf-8"))
        f.close()
    else: print('annotate peaks file: ' + aPPath + ' already exists!')
    return aPPath

def diff_peaks(dir1, dir2, outdir, dist='given', name='', rev=False, print_err=False):
    import subprocess
    import os
    # make output directory if necessary
    if not os.path.isdir(outdir): subprocess.call(["mkdir", outdir])
    # call merge peaks
    mPPath = merge_peaks(dir1, dir2, outdir, dist, name=name)
    # create differential peaks all
    gDPCall = ["getDifferentialPeaks", mPPath,
                        dir1 + "/pooled_tag_dirs",
                        dir2 + "/pooled_tag_dirs"]
    # adjust if it will call reverse peaks against background
    if rev:
        gDPCall.append("-rev")
        dPPath = outdir + '/' +  name + 'diffpeaks_rev.txt'
    else:
        dPPath = outdir + '/' +  name + 'diffpeaks.txt'
    # call differential peaks
    if not os.path.isfile(dPPath):
        print('calling getDifferentialPeaks...')
        p = subprocess.Popen(gDPCall,
                        stdout = subprocess.PIPE,
                        stderr = subprocess.PIPE)
        output, err = p.communicate() #in bytes
        if print_err: print(err.decode('utf-8'))
        # print output
        f = open(dPPath, 'w')
        f.write(output.decode("utf-8"))
        f.close()
    else: print('diff peaks file: ' + dPPath + ' already exists!')
    return dPPath

def find_motifs_genome(peakfile, genome, outdir, size = 200, force=False):
    import subprocess
    import os

    if not os.path.isdir(outdir):
        subprocess.call(["mkdir", outdir])
    
    homerCall = ["findMotifsGenome.pl", peakfile, genome, outdir, "-size", "200"]
    if ((not os.path.isdir(outdir + 'homerResults')) & (not force)):
        print('calling homer...')
        p = subprocess.call(homerCall)
    else: print('homer file already detected, use force=True to overwrite')
    
def clean_sample_IDs(idList):
    import re
    p = re.compile('C57[\w]{0,3}_', re.IGNORECASE)
    tmp = [i.replace("-", "_").strip("mouse_") for i in idList]
    tmp = [p.sub(i, "C57BL6_", re.I) for i in tmp]
    cleanIdList = []
    for i in tmp:
        idParts = i.split("_")
        idParts[0] = idParts[0].upper()
        cleanIdList.append("_".join(idParts))
    return cleanIdList

def run_atac_diff_peaks(dir1, dir2, outdir, dist='given', print_err=False):
    '''Compare the peaks from dir1 and dir1, output basic graphs and a
    pandas data frame with annotations and differential peaks marked.
    
    takes the full path to the directories and output directory
    '''
    import subprocess
    import os
    import pandas
    
    if not os.path.isdir(outdir):
        subprocess.call(["mkdir", outdir])
    samples= [dir1.split('/')[-1], dir2.split('/')[-1]]
    ## Call anno peaks ##
    aPPath = anno_peaks(dir1, dir2, outdir, dist=dist, print_err=print_err)
    ## Get diff peaks ##
    dPPath = diff_peaks(dir1, dir2, outdir, dist=dist, print_err=print_err)
    dP = read_diff_peak_file(dPPath)
    dPPathRev = diff_peaks(dir1, dir2, outdir, dist=dist, print_err=print_err, rev=True)
    dPRev = read_diff_peak_file(dPPathRev)
    # we will plot with out main peak file
    peaks = pd.read_csv(aPPath, sep='\t', comment=None)
    sample_cols = list(peaks.columns.values[-2:])
    peaks.columns.values[0] = 'PeakID'
    peaks = peaks.set_index('PeakID')
    # Annotate main peaks file with whether peaks are differential
    peaks.loc[:,'diffPeak'] = ['nonsig'] * peaks.shape[0]
    peaks.loc[peaks.index.str.contains('|'.join(dP.iloc[:, 0])), 'diffPeak'] = 'up'
    peaks.loc[peaks.index.str.contains('|'.join(dPRev.iloc[:, 0])), 'diffPeak'] = 'dn'

    # add log2 scale for counts for plotting
    peaks.loc[:, 'log2'+samples[0]] = np.log2(peaks.loc[:, sample_cols[0]] + 1)
    peaks.loc[:, 'log2'+samples[1]] = np.log2(peaks.loc[:, sample_cols[1]] + 1)
    f = sns.lmplot(x='log2'+samples[1],
            y='log2'+samples[0],
            data=peaks,
            fit_reg=False,
            hue='diffPeak',
            hue_order=['nonsig', 'up', 'dn'],
            size=8)
    plt.title('Shared ATAC Peaks')
    
    return (peaks, dP, dPRev)

def read_diff_peak_file(dppath, nSkip=18):
    import pandas as pd
    from pandas.parser import CParserError
    
    try:
        dP = pd.read_csv(dppath, sep = '\t', skiprows = nSkip, comment=None)
        return dP
    except CParserError:
        print('skipped incorrect number of lines, check input file')
        return None
    return None

In [4]:
# define sample data frame
samplesDF = pd.DataFrame([s for groups in samples.values() for s in groups], columns = ['subject'])
samplesDF['sample_type'] = ['liver' if 'WholeLiver' in name else 'kupffer' for name in samplesDF.subject]
samplesDF['path'] = [datapath[key] + s for key in datapath.keys() for s in samples[key]]
samplesDF['batch'] = pd.factorize(samplesDF.subject\
             .str.replace('-','_')\
             .str.findall(r'(\d+_\d+_\d+)').str[0])[0]
samplesDF['strain'] = [j.replace('-', '_').split('_')[0].lower() for j in samplesDF.subject]
samplesDF['diet'] = ['amln' if 'AMLN' in name else 'control' for name in samplesDF.subject]
samplesDF['group'] = samplesDF.strain + '_' + samplesDF.diet
samplesDF['group_tissue'] = samplesDF.strain + '_' + samplesDF.diet + '_' + samplesDF.sample_type
samplesDF['exclusion'] = samplesDF.subject.str.contains('|'.join(bad_samples.keys()),
                                                       flags=re.IGNORECASE)

In [5]:
# set colors for the samples
color_dict = {'aj_control':'#fb9a99', 'aj_amln':'#e31a1c',
              'balbc_control':'#a6cee3', 'balbc_amln':'#1f78b4',
              'ncorwt_control':'#b2df8a', 'ncorwt_amln':'#33a02c'}
color_dict_tissue = {'aj_control_kupffer':'#fb9a99', 'aj_amln_kupffer':'#e31a1c',
            'balbc_control_kupffer':'#a6cee3', 'balbc_amln_kupffer':'#1f78b4',
            'ncorwt_control_kupffer':'#b2df8a', 'ncorwt_amln_kupffer':'#33a02c',
            'aj_control_liver':'#fdae6b', 'aj_amln_liver':'#e6550d',
            'balbc_control_liver':'#bcbddc', 'balbc_amln_liver':'#756bb1',
            'ncorwt_control_liver':'#bdbdbd', 'ncorwt_amln_liver':'#737373'}
samplesDF['color'] = [color_dict[group] for group in samplesDF.group]

# write out samplesDF
samplesDF.to_csv(outdir + 'amln_samples.txt', sep='\t')

In [6]:
for i in samplesDF.group.unique():
    analysis = i.replace('-', '_')
    print('analyzing ' + analysis)
    tmp = samplesDF[samplesDF.group == i]
    
    if not (os.path.isdir(outdir + analysis) & os.path.isfile(outdir + analysis + '/optimal_peakfile.txt')):
        #Run IDR with wrapper from Verena
        perlCallList = ['perl', '/home/h1bennet/liverStrains/bin/run_IDR.pl', '-tag_dirs']
        perlCallList.extend(tmp.path)
        perlCallList.extend(['-output_dir', outdir + analysis, '-method', 'atac'])
        subprocess.call(perlCallList)
    
    if not os.path.isfile(outdir + analysis + '/ann_optimal_peakfile.txt'):
        #Run Annotate peaks
        aPCallList = ['annotatePeaks.pl', outdir + analysis + '/optimal_peakfile.txt', 'mm10', '-d']
        aPCallList.extend(tmp.path)
        p = subprocess.Popen(aPCallList, stdout = subprocess.PIPE)
        output = p.stdout.read()
        f = open(outdir + analysis + '/ann_optimal_peakfile.txt', 'w')
        f.write(output.decode("utf-8"))
        f.close()

analyzing aj_amln
analyzing aj_control
analyzing balbc_amln
analyzing balbc_control
analyzing ncorwt_amln
analyzing ncorwt_control


In [17]:
mp = ["mergePeaks", "-d", "200", 
      "aj_amln/optimal_peakfile.txt", "aj_control/optimal_peakfile.txt",
      "balbc_amln/optimal_peakfile.txt", "balbc_control/optimal_peakfile.txt",
      "ncorwt_amln/optimal_peakfile.txt", "ncorwt_control/optimal_peakfile.txt",
      ">", "atac_union.txt"]
print(' '.join(mp))

mergePeaks -d 200 aj_amln/optimal_peakfile.txt aj_control/optimal_peakfile.txt balbc_amln/optimal_peakfile.txt balbc_control/optimal_peakfile.txt ncorwt_amln/optimal_peakfile.txt ncorwt_control/optimal_peakfile.txt > atac_union.txt


From mergePeaks we get a count of how many peaks are in each file:
* aj_amln/optimal_peakfile.txt (33118 total)
* aj_control/optimal_peakfile.txt (55387 total)
* balbc_amln/optimal_peakfile.txt (39842 total)
* balbc_control/optimal_peakfile.txt (33156 total)
* ncorwt_amln/optimal_peakfile.txt (45727 total)
* ncorwt_control/optimal_peakfile.txt (51213 total)

In [27]:
ap = ["annotatePeaks.pl", "atac_union.txt", "mm10", "-noann", "-nogene", '-noadj', "-d"]
ap.extend(samplesDF.path)
ap.extend(['>', 'ann_atac_union.txt'])
print(' '.join(ap))

annotatePeaks.pl atac_union.txt mm10 -noann -nogene -noadj -d /data/mm10/Kupffer/ATAC/aj_Kupffer_ATAC_AMLNDiet_30week_AJ3A_JSS_TDT_16_09_26 /data/mm10/Kupffer/ATAC/aj_Kupffer_ATAC_AMLNDiet_30week_AJ3B_JSS_TDT_16_09_26 /data/mm10/Kupffer/ATAC/aj_Kupffer_ATAC_ControlDiet_30week_AJ1B_JSS_TDT_16_09_26 /data/mm10/Kupffer/ATAC/aj_Kupffer_ATAC_ControlDiet_30week_AJ1C_JSS_TDT_16_09_28 /data/mm10/Kupffer/ATAC/balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3C_JSS_TDT_16_09_26 /data/mm10/Kupffer/ATAC/balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3D_JSS_TDT_16_09_28 /data/mm10/Kupffer/ATAC/balbc_Kupffer_ATAC_ControlDiet_30week_Balb1A_JSS_TDT_16_09_26 /data/mm10/Kupffer/ATAC/balbc_Kupffer_ATAC_ControlDiet_30week_Balb1B_JSS_TDT_16_09_26 /data/mm10/Kupffer/ATAC/NCoRWT_KupfferTotal_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_16_09_19 /data/mm10/Kupffer/ATAC/NCoRWT_KupfferTotal_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_16_09_21 /data/mm10/Kupffer/ATAC/NCoRWT_KupfferTotal_ATAC_ControlDiet_30week_LN134B_JSS_TDT_16_09_21 /data/mm10/K

Total peaks across all samples (from annotatePeaks.pl Total Peaks: 69430)

In [19]:
# read in the ATAC counts data
atac = pd.read_csv(outdir + 'ann_atac_union.txt', sep='\t')
atac.columns.values[0] = 'PeakID'
atac = atac.set_index('PeakID')
counts_mat_k = atac.iloc[:, 6:]
# Column Data Information for DESeq, make sure to drop the samples we are excluding
col_data_k = samplesDF.loc[(samplesDF.sample_type=='kupffer') & ~samplesDF.exclusion,
                           ['subject', 'strain', 'diet', 'sample_type']]
col_data_k = col_data_k.set_index('subject')

In [20]:
# write out data so we can use in R for exploration...
col_data_k.to_csv(outdir + 'deseq_col_data.txt', sep='\t')
counts_mat_k.to_csv(outdir + 'deseq_atac_counts.txt', sep='\t')

### Use HOMER to get diff peaks 

In [42]:
outdir = '/home/h1bennet/liverStrains/results/180103_ATAC/homer/'

In [43]:
diff_peaks(outdir + 'aj_amln', outdir + 'aj_control', outdir, name='aj_amln')
diff_peaks(outdir + 'aj_amln', outdir + 'aj_control', outdir, name='aj_amln_rev', rev=True)
diff_peaks(outdir + 'balbc_amln', outdir + 'balbc_control', outdir, name='balbc_amln')
diff_peaks(outdir + 'balbc_amln', outdir + 'balbc_control', outdir, name='balbc_amln_rev', rev=True)
diff_peaks(outdir + 'ncorwt_amln', outdir + 'ncorwt_control', outdir, name='ncorwt_amln')
diff_peaks(outdir + 'ncorwt_amln', outdir + 'ncorwt_control', outdir, name='ncorwt_amln_rev', rev=True)

calling mergePeaks...
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC/homer//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC/homer//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC/homer//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC/homer//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC/homer//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...


'/home/h1bennet/liverStrains/results/180103_ATAC/homer//ncorwt_amln_revdiffpeaks_rev.txt'

In [40]:
diff_peaks(outdir + 'aj_amln', outdir + 'balbc_amln', outdir, name='aj_balbc_amln_diff_peaks')
diff_peaks(outdir + 'aj_amln', outdir + 'balbc_amln', outdir, name='aj_balbc_amln_diff_peaks_rev', rev=True)
diff_peaks(outdir + 'aj_amln', outdir + 'ncorwt_amln', outdir, name='aj_ncorwt_amln_diff_peaks')
diff_peaks(outdir + 'aj_amln', outdir + 'ncorwt_amln', outdir, name='aj_ncorwt_amln_diff_peaks_rev', rev=True)
diff_peaks(outdir + 'balbc_amln', outdir + 'ncorwt_amln', outdir, name='balbc_ncorwt_amln_diff_peaks')
diff_peaks(outdir + 'balbc_amln', outdir + 'ncorwt_amln', outdir, name='balbc_ncorwt_amln_diff_peaks_rev', rev=True)

merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...


'/home/h1bennet/liverStrains/results/180103_ATAC//balbc_ncorwt_amln_diff_peaks_revdiffpeaks_rev.txt'

In [41]:
diff_peaks(outdir + 'aj_control', outdir + 'balbc_control', outdir, name='aj_balbc_control_diff_peaks')
diff_peaks(outdir + 'aj_control', outdir + 'balbc_control', outdir, name='aj_balbc_control_diff_peaks_rev', rev=True)
diff_peaks(outdir + 'aj_control', outdir + 'ncorwt_control', outdir, name='aj_ncorwt_control_diff_peaks')
diff_peaks(outdir + 'aj_control', outdir + 'ncorwt_control', outdir, name='aj_ncorwt_control_diff_peaks_rev', rev=True)
diff_peaks(outdir + 'balbc_control', outdir + 'ncorwt_control', outdir, name='balbc_ncorwt_control_diff_peaks')
diff_peaks(outdir + 'balbc_control', outdir + 'ncorwt_control', outdir, name='balbc_ncorwt_control_diff_peaks_rev', rev=True)

merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...
merge peaks file /home/h1bennet/liverStrains/results/180103_ATAC//optimal_peakfile_merge.txt already exists!
calling getDifferentialPeaks...


KeyboardInterrupt: 

### RUN DESeq2 analysis of ATAC peaks 

In [21]:
# load extension for running R from python
%load_ext rpy2.ipython

/home/h1bennet/anaconda3/envs/deseq/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: During startup - 
  warnings.warn(x, RRuntimeWarning)
/home/h1bennet/anaconda3/envs/deseq/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Warning messages:

  warnings.warn(x, RRuntimeWarning)
/home/h1bennet/anaconda3/envs/deseq/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 1: package ‘methods’ was built under R version 3.3.3 

  warnings.warn(x, RRuntimeWarning)
/home/h1bennet/anaconda3/envs/deseq/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 2: package ‘datasets’ was built under R version 3.3.3 

  warnings.warn(x, RRuntimeWarning)
/home/h1bennet/anaconda3/envs/deseq/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 3: package ‘utils’ was built under R version 3.3.3 

  warnings.warn(x, RRuntimeWarning)
/home/h1bennet/anaconda3/envs/deseq/lib/python3.6

#### Run full interaction model 

In [22]:
%%R -i counts_mat_k,col_data_k,outdir
# load DESeq
library("DESeq2")
library("data.table")

# remove chrM from analysis (no NCorWT peaks map to it - problematic)
counts_mat_k <- counts_mat_k[!(rownames(counts_mat_k) %like% 'chrM'), ]
colnames(counts_mat_k) <- lapply(strsplit(colnames(counts_mat_k), '\\.'), `[[`, 6)

col_data_k$diet <- relevel(col_data_k$diet, ref='control')
col_data_k$strain <- relevel(col_data_k$strain, ref='ncorwt')

# names(counts_mat_k) <- NULL

#create DESeq2 object
dds <- DESeqDataSetFromMatrix(countData = counts_mat_k,
                             colData = col_data_k,
                             design = ~ strain + diet + strain:diet)

# run DESeq with full model
dds <- DESeq(dds)

# diet effect for ncorwt
res <- results(dds, contrast=c('diet', 'amln', 'control'), alpha = 0.05)
res_ordered <- res[order(res$padj), ]
write.csv(as.data.frame(res_ordered), 
          file=paste0(outdir, "/ncorwt_amln_atac_deseq_results.txt"))

# diet effect for balbc
res <- results(dds, contrast=list( c('diet_amln_vs_control','strainbalbc.dietamln') ), alpha = 0.05)
res_ordered <- res[order(res$padj), ]
write.csv(as.data.frame(res_ordered), 
          file=paste0(outdir, "/balbc_amln_atac_deseq_results.txt"))

# diet effect for aj
res <- results(dds, contrast=list( c('diet_amln_vs_control','strainaj.dietamln') ), alpha = 0.05)
res_ordered <- res[order(res$padj), ]
write.csv(as.data.frame(res_ordered), 
          file=paste0(outdir, "/aj_amln_atac_deseq_results.txt"))

# interaction term for condition effect between balbc and ncorwt
res <- results(dds, name='strainbalbc.dietamln', alpha = 0.05)
res_ordered <- res[order(res$padj), ]
write.csv(as.data.frame(res_ordered), 
          file=paste0(outdir, "/balbc_ncorwt_amln_interaction_atac_deseq_results.txt"))

# interaction term for condition effect between aj and ncorwt
res <- results(dds, name='strainaj.dietamln', alpha = 0.05)
res_ordered <- res[order(res$padj), ]
write.csv(as.data.frame(res_ordered), 
          file=paste0(outdir, "/aj_ncorwt_amln_interaction_atac_deseq_results.txt"))

# interaction term for condition effect between balbc and aj
res <- results(dds, contrast=list("strainbalbc.dietamln", "strainaj.dietamln"), alpha = 0.05)
res_ordered <- res[order(res$padj), ]
write.csv(as.data.frame(res_ordered), 
          file=paste0(outdir, "/balbc_aj_amln_interaction_atac_deseq_results.txt"))

/home/h1bennet/anaconda3/envs/deseq/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: S4Vectors

  warnings.warn(x, RRuntimeWarning)
/home/h1bennet/anaconda3/envs/deseq/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: stats4

  warnings.warn(x, RRuntimeWarning)
/home/h1bennet/anaconda3/envs/deseq/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: BiocGenerics

  warnings.warn(x, RRuntimeWarning)
/home/h1bennet/anaconda3/envs/deseq/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: parallel

  warnings.warn(x, RRuntimeWarning)
/home/h1bennet/anaconda3/envs/deseq/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attaching package: ‘BiocGenerics’


  warnings.warn(x, RRuntimeWarning)
/home/h1bennet/anaconda3/envs/deseq/lib/python3.6/site-packages/rp

#### Run simple pairwise model 

In [26]:
%%R -i counts_mat_k,col_data_k,outdir
# load DESeq
library("DESeq2")
library("data.table")

# remove chrM from analysis (no NCorWT peaks map to it - problematic)
counts_mat_k <- counts_mat_k[!(rownames(counts_mat_k) %like% 'chrM'), ]
colnames(counts_mat_k) <- lapply(strsplit(colnames(counts_mat_k), '\\.'), `[[`, 6)

col_data_k$diet <- relevel(col_data_k$diet, ref='control')
col_data_k$strain <- relevel(col_data_k$strain, ref='ncorwt')
col_data_k$straindiet = as.factor(paste0(col_data_k$strain, '_', col_data_k$diet))
col_data_k$straindiet <- relevel(col_data_k$straindiet, ref='ncorwt_amln')

# names(counts_mat_k) <- NULL

#create DESeq2 object
ddsRed <- DESeqDataSetFromMatrix(countData = counts_mat_k,
                             colData = col_data_k,
                             design = ~ straindiet)

# run DESeq with full model
ddsRed <- DESeq(ddsRed)

# diet effect for ncorwt
resRed <- results(ddsRed, contrast=c('straindiet','ncorwt_amln','ncorwt_control'), alpha = 0.1)
resRed_ordered <- resRed[order(resRed$padj), ]
write.csv(as.data.frame(resRed_ordered), 
          file=paste0(outdir, "/ncorwt_amln_atac_deseq_pairwise_results.txt"))

# diet effect for balbc
resRed <- results(ddsRed, contrast=c('straindiet','balbc_amln','balbc_control'), alpha = 0.1)
resRed_ordered <- resRed[order(resRed$padj), ]
write.csv(as.data.frame(resRed_ordered), 
          file=paste0(outdir, "/balbc_amln_atac_deseq_pairwise_results.txt"))

# diet effect for aj
resRed <- results(ddsRed, contrast=c('straindiet','aj_amln','aj_control'), alpha = 0.1)
resRed_ordered <- resRed[order(resRed$padj), ]
write.csv(as.data.frame(resRed_ordered), 
          file=paste0(outdir, "/aj_amln_atac_deseq_pairwise_results.txt"))

# compare aj and ncor controls
resRed <- results(ddsRed, contrast=c('straindiet','aj_control','ncorwt_control'), alpha = 0.1)
resRed_ordered <- resRed[order(resRed$padj), ]
write.csv(as.data.frame(resRed_ordered), 
          file=paste0(outdir, "/aj_ncorwt_control_atac_deseq_pairwise_results.txt"))

# compare aj and balbc controls
resRed <- results(ddsRed, contrast=c('straindiet','aj_control','balbc_control'), alpha = 0.1)
resRed_ordered <- resRed[order(resRed$padj), ]
write.csv(as.data.frame(resRed_ordered), 
          file=paste0(outdir, "/aj_balbc_control_atac_deseq_pairwise_results.txt"))

# compare balbc and ncor controls
resRed <- results(ddsRed, contrast=c('straindiet','balbc_control','ncorwt_control'), alpha = 0.1)
resRed_ordered <- resRed[order(resRed$padj), ]
write.csv(as.data.frame(resRed_ordered), 
          file=paste0(outdir, "/balbc_ncorwt_control_atac_deseq_pairwise_results.txt"))

# compare aj and ncor amln
resRed <- results(ddsRed, contrast=c('straindiet','aj_amln','ncorwt_amln'), alpha = 0.1)
resRed_ordered <- resRed[order(resRed$padj), ]
write.csv(as.data.frame(resRed_ordered), 
          file=paste0(outdir, "/aj_ncorwt_amln_atac_deseq_pairwise_results.txt"))

# compare aj and balbc amln
resRed <- results(ddsRed, contrast=c('straindiet','aj_amln','balbc_amln'), alpha = 0.1)
resRed_ordered <- resRed[order(resRed$padj), ]
write.csv(as.data.frame(resRed_ordered), 
          file=paste0(outdir, "/aj_balbc_amln_atac_deseq_pairwise_results.txt"))

# compare balbc and ncor amln
resRed <- results(ddsRed, contrast=c('straindiet','balbc_amln','ncorwt_amln'), alpha = 0.1)
resRed_ordered <- resRed[order(resRed$padj), ]
write.csv(as.data.frame(resRed_ordered), 
          file=paste0(outdir, "/balbc_ncorwt_amln_atac_deseq_pairwise_results.txt"))

We want to examine these peaks in the genome browser, so we need to be able to pull up their coordinates

In [7]:
outdir = '/home/h1bennet/liverStrains/results/180103_ATAC/'

In [13]:
atac_union = pd.read_csv(outdir + "atac_union.txt", header=0, index_col=0, sep='\t')
atac_union['browser'] = atac_union.chr + ':' + atac_union.start.map(str) + '-' + atac_union.end.map(str)

In [9]:
# read in the deseq full model results
aj_amln = pd.read_csv(outdir + "deseq/aj_amln_atac_deseq_results.txt")
ncorwt_amln = pd.read_csv(outdir + "deseq/ncorwt_amln_atac_deseq_results.txt")
balb_amln = pd.read_csv(outdir + "deseq/balbc_amln_atac_deseq_results.txt")
aj_balb = pd.read_csv(outdir + "deseq/balbc_aj_amln_interaction_atac_deseq_results.txt")
aj_ncor = pd.read_csv(outdir + "deseq/aj_ncorwt_amln_interaction_atac_deseq_results.txt")
ncorwt_balb = pd.read_csv(outdir + "deseq/balbc_ncorwt_amln_interaction_atac_deseq_results.txt")

In [10]:
# read in the pairwise model results
aj_amln_p = pd.read_csv(outdir + "deseq/aj_amln_atac_deseq_pairwise_results.txt")
balb_amln_p = pd.read_csv(outdir + "deseq/balbc_amln_atac_deseq_pairwise_results.txt")
ncorwt_amln_p = pd.read_csv(outdir + "deseq/ncorwt_amln_atac_deseq_pairwise_results.txt")
aj_ncorwt_control = pd.read_csv(outdir + "deseq/aj_balbc_control_atac_deseq_pairwise_results.txt")
aj_balbc_control = pd.read_csv(outdir + "deseq/aj_ncorwt_control_atac_deseq_pairwise_results.txt")
balbc_ncorwt_control = pd.read_csv(outdir + "deseq/balbc_ncorwt_control_atac_deseq_pairwise_results.txt")
aj_ncorwt_amln = pd.read_csv(outdir + "deseq/aj_balbc_amln_atac_deseq_pairwise_results.txt")
aj_balbc_amln = pd.read_csv(outdir + "deseq/aj_ncorwt_amln_atac_deseq_pairwise_results.txt")
balbc_ncorwt_amln = pd.read_csv(outdir + "deseq/balbc_ncorwt_amln_atac_deseq_pairwise_results.txt")

In [29]:
aj_amln.loc[aj_amln.padj < 0.1, ]

,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,Merged-chr17-34264742-3,21.862139,2.888149,0.572310,5.046477,4.500302e-07,0.027875
1,Merged-chr6-47650218-2,14.167452,3.490617,0.707380,4.934573,8.032623e-07,0.027875
2,Merged-chr7-19305092-2,16.956990,2.907727,0.616900,4.713446,2.435625e-06,0.056348
3,Merged-chr17-45556031-1,12.849818,3.211468,0.705501,4.552040,5.312813e-06,0.092184


In [28]:
atac_union.loc[aj_amln.loc[aj_amln.padj < 0.1, 'Unnamed: 0'], ]

,chr,start,end,strand,Stat,Parent files,Total subpeaks,aj_amln/optimal_peakfile.txt,aj_control/optimal_peakfile.txt,balbc_amln/optimal_peakfile.txt,balbc_control/optimal_peakfile.txt,ncorwt_amln/optimal_peakfile.txt,ncorwt_control/optimal_peakfile.txt,browser
#name (cmd = mergePeaks -d 200 aj_amln/optimal_peakfile.txt aj_control/optimal_peakfile.txt balbc_amln/optimal_peakfile.txt balbc_control/optimal_peakfile.txt ncorwt_amln/optimal_peakfile.txt ncorwt_control/optimal_peakfile.txt),,,,,,,,,,,,,,
Merged-chr17-34264742-3,chr17,34264558,34264899,+,14.966666,aj_amln/optimal_peakfile.txt|balbc_control/opt...,3,chr17-943,NaN,NaN,chr17-1390,chr17-2375,NaN,chr17:34264558-34264899
Merged-chr6-47650218-2,chr6,47650102,47650334,+,13.500000,aj_amln/optimal_peakfile.txt|balbc_amln/optima...,2,chr6-729,NaN,chr6-2019,NaN,NaN,NaN,chr6:47650102-47650334
Merged-chr7-19305092-2,chr7,19304976,19305208,+,19.950001,aj_amln/optimal_peakfile.txt|ncorwt_control/op...,2,chr7-1176,NaN,NaN,NaN,NaN,chr7-2536,chr7:19304976-19305208
Merged-chr17-45556031-1,chr17,45555931,45556131,+,18.700001,aj_amln/optimal_peakfile.txt,1,chr17-851,NaN,NaN,NaN,NaN,NaN,chr17:45555931-45556131


In [30]:
# aj diff peaks
aj_amln = pd.read_csv(outdir + "/aj_amln_atac_deseq_results.txt")
aj_balb = pd.read_csv(outdir + "/balbc_aj_amln_interaction_atac_deseq_results.txt")
aj_ncor = pd.read_csv(outdir + "/aj_ncorwt_amln_interaction_atac_deseq_results.txt")

FileNotFoundError: File b'/home/h1bennet/liverStrains/results/180103_ATAC//aj_amln_atac_deseq_results.txt' does not exist

In [31]:
(aj_amln.padj <= 0.1).value_counts()

False    69401
True         4
Name: padj, dtype: int64

In [20]:
# combine into one DF
aj = aj_amln.merge(aj_balb.merge(aj_ncor, how='outer', on='Unnamed: 0',
                                    suffixes=('_aj_balb', '_aj_ncor')),
                      how='outer', on='Unnamed: 0')
# get aj specific peaks
aj_balb_spec = ((np.abs(aj.log2FoldChange) >= 1) & (aj.padj <=0.1) & (aj.padj_aj_balb <= 0.1))
aj_balb_spec.value_counts()

False    60785
dtype: int64

In [21]:
aj_ncor_spec = ((np.abs(aj.log2FoldChange) >= 1) & (aj.padj <=0.1) & (aj.padj_aj_ncor <= 0.1))
aj_ncor_spec.value_counts()

False    60785
dtype: int64

In [22]:
((aj_balb_spec) & (aj_ncor_spec)).value_counts()


False    60785
dtype: int64

In [10]:
ncorwt_amln = pd.read_csv(outdir + "/ncorwt_amln_atac_deseq_results.txt")
ncorwt_aj = pd.read_csv(outdir + "/aj_ncorwt_amln_interaction_atac_deseq_results.txt")
ncorwt_balb = pd.read_csv(outdir + "/balbc_ncorwt_amln_interaction_atac_deseq_results.txt")

In [11]:
(ncorwt_amln.padj <= 0.1).value_counts()

False    60675
True       110
Name: padj, dtype: int64

In [23]:
# combine into one DF
ncorwt = ncorwt_amln.merge(ncorwt_balb.merge(ncorwt_aj, how='outer', on='Unnamed: 0',
                                             suffixes=('_ncorwt_balb', '_ncorwt_aj')),
                           how='outer', on='Unnamed: 0')

In [24]:
ncorwt_balb_spec = ((np.abs(ncorwt.log2FoldChange) >= 1) & (ncorwt.padj <=0.1) & (ncorwt.padj_ncorwt_balb <= 0.1))
ncorwt_balb_spec.value_counts()

False    60784
True         1
dtype: int64

In [25]:
ncorwt_aj_spec = ((np.abs(ncorwt.log2FoldChange) >= 1) & (ncorwt.padj <=0.1) & (ncorwt.padj_ncorwt_aj <= 0.1))
ncorwt_aj_spec.value_counts()

False    60785
dtype: int64

In [14]:
balb_amln = pd.read_csv(outdir + "/balbc_amln_atac_deseq_results.txt")

In [15]:
(balb_amln.padj <= 0.1).value_counts()

False    60785
Name: padj, dtype: int64